# Customer Sales Representative
    Hey I am a customer sales representative. My only work here is to entertain our beloved clients and in results I get a pay raise. (Party) 

In [1]:
from pymongo import MongoClient
import pymongo


connectionString = "mongodb://192.168.18.4:27017,192.168.18.4:27018,192.168.18.4:27019/magic_shop?replicaSet=mongodb-replicaset";
client           = MongoClient(connectionString)
db               = client.magic_shop        # test is my database
col              = db.req                   # Here spam is my collection

# Listening to the Insert Stream

In [ ]:
import socket
machine_name = socket.gethostname()
from datetime import datetime
import time

def processing_request(request):
    '''
    Description:
        Get the string, takes its length and run a loop 10 times of string length and set the 
        response to processed.
        
    Input:
        request  (str) : Random String
        
    Output:
        Set value back to the mongo db
            response: Processed
    '''
    for i in range(0,len(request)*10):
        pass
    
    

try:
    with db.req.watch([{'$match': {'operationType': 'insert'}}]) as stream:
        for values in stream:
            request_id = values['fullDocument']['request_id']
            request    = values['fullDocument']['request']
            myquery    = { "request_id": request_id }
            
            # Checking if not processed by other replica (Blocking System)
            if len(list(col.find({ 'request_id': request_id, 'CSR_NAME' : { "$exists": False}}))):
                newvalues  = { "$set": { "CSR_NAME": machine_name, "response": "Processing", "processing_time":datetime.today()} }

                # CSR Responding to the client
                col.update_one(myquery, newvalues)
                print(request_id)
                print(f"Processing {request_id}")
                # Now Processing Request
                processing_request(request)
                # Updating that work is done

                myquery   = { "request_id": request_id }
                newvalues = { "$set": {"response": "Request Processed Have a nice Day sir!", 'response_time':datetime.today()} }
                # CSR Responding to the client
                col.update_one(myquery, newvalues)
            
except pymongo.errors.PyMongoError:
    # The ChangeStream encountered an unrecoverable error or the
    # resume attempt failed to recreate the cursor.
    logging.error('...')

38634
Processing 38634
77998
Processing 77998
11342
Processing 11342
14565
Processing 14565
17425
Processing 17425
16576
Processing 16576
81226
Processing 81226
77268
Processing 77268
31480
Processing 31480
51178
Processing 51178
25174
Processing 25174
25544
Processing 25544
23130
Processing 23130
73680
Processing 73680
94201
Processing 94201
59662
Processing 59662
11533
Processing 11533
68855
Processing 68855
76606
Processing 76606
57567
Processing 57567
96237
Processing 96237
99391
Processing 99391
20701
Processing 20701
32287
Processing 32287
76889
Processing 76889
15436
Processing 15436
49377
Processing 49377
18635
Processing 18635
67497
Processing 67497
76589
Processing 76589
60449
Processing 60449
33506
Processing 33506
94554
Processing 94554
56645
Processing 56645
3144
Processing 3144
92947
Processing 92947
55280
Processing 55280
43360
Processing 43360
36252
Processing 36252
75438
Processing 75438
32726
Processing 32726
41990
Processing 41990
58007
Processing 58007
89203
Process

71027
Processing 71027
23576
Processing 23576
4750
Processing 4750
20409
Processing 20409
59946
Processing 59946
63528
Processing 63528
99368
Processing 99368
83378
Processing 83378
78406
Processing 78406
73649
Processing 73649
39021
Processing 39021
79011
Processing 79011
31912
Processing 31912
74721
Processing 74721
70457
Processing 70457
59387
Processing 59387
24623
Processing 24623
4516
Processing 4516
67462
Processing 67462
1289
Processing 1289
62702
Processing 62702
11010
Processing 11010
44172
Processing 44172
94947
Processing 94947
15582
Processing 15582
96345
Processing 96345
48087
Processing 48087
36048
Processing 36048
19427
Processing 19427
8754
Processing 8754
38714
Processing 38714
38786
Processing 38786
15122
Processing 15122
66287
Processing 66287
65653
Processing 65653
77675
Processing 77675
58832
Processing 58832
50045
Processing 50045
49621
Processing 49621
56106
Processing 56106
56157
Processing 56157
30861
Processing 30861
36938
Processing 36938
61183
Processing 61

87976
Processing 87976
92091
Processing 92091
91027
Processing 91027
95111
Processing 95111
74678
Processing 74678
71750
Processing 71750
59050
Processing 59050
81779
Processing 81779
6849
Processing 6849
84387
Processing 84387
74217
Processing 74217
39519
Processing 39519
4163
Processing 4163
67604
Processing 67604
50595
Processing 50595
9571
Processing 9571
32504
Processing 32504
61352
Processing 61352
31352
Processing 31352
75310
Processing 75310
56240
Processing 56240
2503
Processing 2503
15453
Processing 15453
78541
Processing 78541
67386
Processing 67386
66732
Processing 66732
89292
Processing 89292
29580
Processing 29580
11916
Processing 11916
81070
Processing 81070
82703
Processing 82703
85885
Processing 85885
76058
Processing 76058
35216
Processing 35216
91773
Processing 91773
1648
Processing 1648
26614
Processing 26614
14757
Processing 14757
81603
Processing 81603
68612
Processing 68612
38708
Processing 38708
39794
Processing 39794
61026
Processing 61026
91639
Processing 9163

38890
Processing 38890
7895
Processing 7895
33521
Processing 33521
58460
Processing 58460
49602
Processing 49602
9402
Processing 9402
87726
Processing 87726
10600
Processing 10600
90501
Processing 90501
88930
Processing 88930
93193
Processing 93193
66628
Processing 66628
18357
Processing 18357
80554
Processing 80554
28925
Processing 28925
28245
Processing 28245
62868
Processing 62868
16935
Processing 16935
26098
Processing 26098
94826
Processing 94826
93877
Processing 93877
48909
Processing 48909
46695
Processing 46695
96798
Processing 96798
98299
Processing 98299
31406
Processing 31406
44711
Processing 44711
58237
Processing 58237
48594
Processing 48594
52854
Processing 52854
46390
Processing 46390
14224
Processing 14224
70890
Processing 70890
56742
Processing 56742
11215
Processing 11215
72685
Processing 72685
11057
Processing 11057
51078
Processing 51078
4084
Processing 4084
97671
Processing 97671
99103
Processing 99103
88003
Processing 88003
63362
Processing 63362
5884
Processing 5

72791
Processing 72791
88416
Processing 88416
63097
Processing 63097
41490
Processing 41490
47532
Processing 47532
67282
Processing 67282
68154
Processing 68154
41147
Processing 41147
73171
Processing 73171
81912
Processing 81912
3795
Processing 3795
35984
Processing 35984
75132
Processing 75132
7033
Processing 7033
98443
Processing 98443
76118
Processing 76118
2784
Processing 2784
22846
Processing 22846
70182
Processing 70182
51521
Processing 51521
45023
Processing 45023
32434
Processing 32434
86838
Processing 86838
57780
Processing 57780
95129
Processing 95129
79864
Processing 79864
14733
Processing 14733
89196
Processing 89196
21160
Processing 21160
29
Processing 29
52192
Processing 52192
55623
Processing 55623
14410
Processing 14410
63716
Processing 63716
83331
Processing 83331
44543
Processing 44543
25573
Processing 25573
41002
Processing 41002
38421
Processing 38421
89260
Processing 89260
24886
Processing 24886
60858
Processing 60858
24412
Processing 24412
45206
Processing 45206


...

31903
Processing 31903
58036
Processing 58036
26444
Processing 26444
32758
Processing 32758
77682
Processing 77682
72649
Processing 72649
24727
Processing 24727
70165
Processing 70165
98299
Processing 98299
31196
Processing 31196
97908
Processing 97908
88292
Processing 88292
97544
Processing 97544
63185
Processing 63185
22677
Processing 22677
98626
Processing 98626
56850
Processing 56850
70629
Processing 70629
33264
Processing 33264
66904
Processing 66904
66999
Processing 66999
11335
Processing 11335
67071
Processing 67071
97367
Processing 97367
16084
Processing 16084
825
Processing 825
6162
Processing 6162
89899
Processing 89899
10630
Processing 10630
36436
Processing 36436
1204
Processing 1204
57292
Processing 57292
86134
Processing 86134
16805
Processing 16805
51678
Processing 51678
24844
Processing 24844
85585
Processing 85585
60006
Processing 60006
33759
Processing 33759
97416
Processing 97416
98800
Processing 98800
50167
Processing 50167
63690
Processing 63690
62503
Processing 62

81461
Processing 81461
38536
Processing 38536
89099
Processing 89099
51496
Processing 51496
11791
Processing 11791
45726
Processing 45726
29083
Processing 29083
62826
Processing 62826
51338
Processing 51338
62405
Processing 62405
1726
Processing 1726
27579
Processing 27579
49712
Processing 49712
2642
Processing 2642
45070
Processing 45070
74252
Processing 74252
12431
Processing 12431
78878
Processing 78878
31544
Processing 31544
46483
Processing 46483
18554
Processing 18554
41088
Processing 41088
60450
Processing 60450
79920
Processing 79920
15807
Processing 15807
19962
Processing 19962
27373
Processing 27373
33071
Processing 33071
10032
Processing 10032
96848
Processing 96848
32826
Processing 32826
77363
Processing 77363
18886
Processing 18886
24019
Processing 24019
4237
Processing 4237
10763
Processing 10763
1276
Processing 1276
69918
Processing 69918
70577
Processing 70577
13010
Processing 13010
306
Processing 306
40778
Processing 40778
55382
Processing 55382
48130
Processing 48130


52523
Processing 52523
25545
Processing 25545
15281
Processing 15281
17429
Processing 17429
5940
Processing 5940
15583
Processing 15583
6437
Processing 6437
66416
Processing 66416
20406
Processing 20406
49023
Processing 49023
17267
Processing 17267
20934
Processing 20934
59946
Processing 59946
58325
Processing 58325
68453
Processing 68453
21054
Processing 21054
15857
Processing 15857
65516
Processing 65516
24284
Processing 24284
6767
Processing 6767
83802
Processing 83802
86053
Processing 86053
7903
Processing 7903
11950
Processing 11950
13368
Processing 13368
11135
Processing 11135
60830
Processing 60830
26211
Processing 26211
25315
Processing 25315
63402
Processing 63402
11993
Processing 11993
32640
Processing 32640
12856
Processing 12856
8039
Processing 8039
45192
Processing 45192
9682
Processing 9682
53108
Processing 53108
6397
Processing 6397
2700
Processing 2700
83744
Processing 83744
28234
Processing 28234
23884
Processing 23884
83825
Processing 83825
80638
Processing 80638
4726

49812
Processing 49812
85170
Processing 85170
12474
Processing 12474
70648
Processing 70648
48408
Processing 48408
17391
Processing 17391
60504
Processing 60504
49084
Processing 49084
6706
Processing 6706
67638
Processing 67638
16792
Processing 16792
85535
Processing 85535
78836
Processing 78836
20518
Processing 20518
66416
Processing 66416
605
Processing 605
47740
Processing 47740
8830
Processing 8830
85604
Processing 85604
94908
Processing 94908
34029
Processing 34029
42328
Processing 42328
30946
Processing 30946
82685
Processing 82685
19824
Processing 19824
43611
Processing 43611
95052
Processing 95052
50278
Processing 50278
16371
Processing 16371
9465
Processing 9465
77588
Processing 77588
45118
Processing 45118
95933
Processing 95933
74131
Processing 74131
46127
Processing 46127
41717
Processing 41717
26611
Processing 26611
92397
Processing 92397
3874
Processing 3874
64609
Processing 64609
48431
Processing 48431
67660
Processing 67660
46239
Processing 46239
10894
Processing 10894


10474
Processing 10474
68367
Processing 68367
63736
Processing 63736
42674
Processing 42674
96304
Processing 96304
7901
Processing 7901
53771
Processing 53771
22313
Processing 22313
78398
Processing 78398
15590
Processing 15590
35801
Processing 35801
81779
Processing 81779
70698
Processing 70698
60871
Processing 60871
66133
Processing 66133
73
Processing 73
61567
Processing 61567
24344
Processing 24344
84163
Processing 84163
31783
Processing 31783
852
Processing 852
82304
Processing 82304
78900
Processing 78900
21215
Processing 21215
46286
Processing 46286
26095
Processing 26095
12908
Processing 12908
75584
Processing 75584
23775
Processing 23775
76920
Processing 76920
78997
Processing 78997
21834
Processing 21834
43776
Processing 43776
22819
Processing 22819
22671
Processing 22671
51233
Processing 51233
19870
Processing 19870
69785
Processing 69785
11702
Processing 11702
65460
Processing 65460
36808
Processing 36808
99785
Processing 99785
10845
Processing 10845
56699
Processing 56699
